In [3]:
import pandas as pd


In [4]:
imdb1 = pd.read_csv("zippedData/imdb.name.basics.csv.gz")
imdb2 = pd.read_csv("zippedData/imdb.title.akas.csv.gz")
imdb3 = pd.read_csv("zippedData/imdb.title.basics.csv.gz")
imdb4 = pd.read_csv("zippedData/imdb.title.crew.csv.gz")
imdb5 = pd.read_csv("zippedData/imdb.title.principals.csv.gz")
imdb6 = pd.read_csv("zippedData/imdb.title.ratings.csv.gz")

In [3]:
rt1 = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep='\t', encoding = 'unicode_escape')
rt2 = pd.read_csv("zippedData/rt.reviews.tsv.gz", sep='\t', encoding = 'unicode_escape')
tn = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

In [4]:
bom = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
tmbd = pd.read_csv("zippedData/tmdb.movies.csv.gz")

In [5]:
print(imdb1.shape)
imdb1.head()

(606648, 6)


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [6]:
imdb1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
nconst                606648 non-null object
primary_name          606648 non-null object
birth_year            82736 non-null float64
death_year            6783 non-null float64
primary_profession    555308 non-null object
known_for_titles      576444 non-null object
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


In [7]:
imdb2.head()

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


In [8]:
imdb2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
title_id             331703 non-null object
ordering             331703 non-null int64
title                331703 non-null object
region               278410 non-null object
language             41715 non-null object
types                168447 non-null object
attributes           14925 non-null object
is_original_title    331678 non-null float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [9]:
imdb3.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [10]:
imdb3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
tconst             146144 non-null object
primary_title      146144 non-null object
original_title     146123 non-null object
start_year         146144 non-null int64
runtime_minutes    114405 non-null float64
genres             140736 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [11]:
imdb4.head()

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


In [12]:
imdb4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
tconst       146144 non-null object
directors    140417 non-null object
writers      110261 non-null object
dtypes: object(3)
memory usage: 3.3+ MB


In [13]:
imdb5.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [14]:
imdb5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
tconst        1028186 non-null object
ordering      1028186 non-null int64
nconst        1028186 non-null object
category      1028186 non-null object
job           177684 non-null object
characters    393360 non-null object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB


In [15]:
imdb6.head()

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [16]:
imdb6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
tconst           73856 non-null object
averagerating    73856 non-null float64
numvotes         73856 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [17]:
rt1.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [18]:
rt2.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [19]:
print(tn.info())
tn.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB
None


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [20]:
print(bom.info())
bom.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
title             3387 non-null object
studio            3382 non-null object
domestic_gross    3359 non-null float64
foreign_gross     2037 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [21]:
tmbd.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


# IMDB Web Scraped Data

In [36]:
# imdb2
print("2. " + imdb2.columns)
print("3. " + imdb3.columns)
print("4. " + imdb4.columns)
print("5. " + imdb5.columns)
print("6. " + imdb6.columns)

Index(['2. title_id', '2. ordering', '2. title', '2. region', '2. language',
       '2. types', '2. attributes', '2. is_original_title'],
      dtype='object')
Index(['3. tconst', '3. primary_title', '3. original_title', '3. start_year',
       '3. runtime_minutes', '3. genres'],
      dtype='object')
Index(['4. tconst', '4. directors', '4. writers'], dtype='object')
Index(['5. tconst', '5. ordering', '5. nconst', '5. category', '5. job',
       '5. characters'],
      dtype='object')
Index(['6. tconst', '6. averagerating', '6. numvotes'], dtype='object')


In [5]:
# cleaning primary keys in imdb dfs
imdb2['title_id'] = imdb2['title_id'].str.strip()
imdb3['tconst'] = imdb3['tconst'].str.strip()
imdb4['tconst'] = imdb4['tconst'].str.strip()
imdb5['tconst'] = imdb5['tconst'].str.strip()
imdb6['tconst'] = imdb6['tconst'].str.strip()

In [6]:
# dropping duplicates
imdb2.drop_duplicates(subset=['title_id'], inplace=True)

In [7]:
imdb3.drop_duplicates(subset=['tconst'], inplace=True)
imdb4.drop_duplicates(subset=['tconst'], inplace=True)
imdb5.drop_duplicates(subset=['tconst'], inplace=True)
imdb6.drop_duplicates(subset=['tconst'], inplace=True)

In [8]:
# importing web scrapes
web_scrape1 = pd.read_csv('imdb_monetary_data_16590.csv')
web_scrape1.columns
web_scrape1.set_index(['title_id'], inplace=True)

In [9]:
web_scrape2 = pd.read_csv('imdb_monetary_data_16591_to_25214_.csv')
web_scrape2.columns
web_scrape2.set_index(['title_id'], inplace=True)

In [10]:
# concatenating webscraped data
web_scrape = pd.concat([web_scrape1, web_scrape2])
print(web_scrape.columns, end='\n\n')
print(web_scrape.shape, end='\n\n')
print(web_scrape.isna().sum())

Index(['Unnamed: 0', 'Country', 'Budget', 'Opening Weekend USA', 'Gross USA',
       'Cumulative Worldwide Gross', 'Runtime'],
      dtype='object')

(25214, 7)

Unnamed: 0                        0
Country                          35
Budget                        17454
Opening Weekend USA           23413
Gross USA                     23322
Cumulative Worldwide Gross    20206
Runtime                        6397
dtype: int64


In [11]:
# dropping unnamed column
web_scrape.drop(['Unnamed: 0'], axis=1, inplace=True)
web_scrape

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime
title_id,,,,,,
tt0369610,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min
tt0381957,UK,NaN,NaN,NaN,NaN,NaN
tt0401729,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min
tt10010134,France,NaN,NaN,NaN,NaN,52 min
tt10027708,Australia,NaN,NaN,NaN,NaN,94 min
...,...,...,...,...,...,...
tt3328050,USA,NaN,NaN,NaN,NaN,69 min
tt3329964,Croatia,"EUR3,000",NaN,NaN,NaN,70 min
tt3330812,USA,NaN,NaN,NaN,NaN,73 min


In [12]:
# cleaning title_id (index)
web_scrape.index = web_scrape.index.str.strip()

In [13]:
# merging imdb2
df = web_scrape.merge(imdb2, left_on=web_scrape.index, right_on=['title_id'])
df

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime,title_id,ordering,title,region,language,types,attributes,is_original_title
0,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,UK,NaN,NaN,NaN,NaN,NaN,tt0381957,1,Cloud Cuckoo Land,GB,NaN,NaN,NaN,0.0
2,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min,tt0401729,10,Джон Картер: Мiж двох свiтiв,UA,NaN,NaN,NaN,0.0
3,France,NaN,NaN,NaN,NaN,52 min,tt10010134,1,Versailles Rediscovered - The Sun King's Vanis...,NaN,NaN,original,NaN,1.0
4,Australia,NaN,NaN,NaN,NaN,94 min,tt10027708,1,Miguelito - Canto a Borinquen,NaN,NaN,original,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25209,USA,NaN,NaN,NaN,NaN,69 min,tt3328050,1,Moving from Emptiness: The Life and Art of a Z...,US,NaN,NaN,NaN,0.0
25210,Croatia,"EUR3,000",NaN,NaN,NaN,70 min,tt3329964,1,Glazbena kutija,HR,NaN,NaN,NaN,0.0
25211,USA,NaN,NaN,NaN,NaN,73 min,tt3330812,1,Night Terrors,US,NaN,NaN,NaN,0.0
25212,Costa Rica,NaN,NaN,NaN,NaN,125 min,tt3331178,1,La Musa es la Montaña,CR,NaN,NaN,NaN,0.0


In [14]:
df = df.merge(imdb3, left_on=['title_id'], right_on=['tconst'])

In [15]:
df = df.merge(imdb4, left_on=['title_id'], right_on=['tconst'])

In [16]:
df = df.merge(imdb5, left_on=['title_id'], right_on=['tconst'])

In [17]:
df = df.merge(imdb6, how='left', left_on=['title_id'], right_on=['tconst'])

In [18]:
df

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime,title_id,ordering_x,title,region,...,writers,tconst_x,ordering_y,nconst,category,job,characters,tconst_y,averagerating,numvotes
0,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min,tt0369610,10,Джурасик свят,BG,...,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",tt0369610,10,nm0189777,producer,producer,NaN,tt0369610,7.0,539338.0
1,UK,NaN,NaN,NaN,NaN,NaN,tt0381957,1,Cloud Cuckoo Land,GB,...,"nm1458371,nm1461088",tt0381957,10,nm1108038,composer,NaN,NaN,tt0381957,7.6,23.0
2,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min,tt0401729,10,Джон Картер: Мiж двох свiтiв,UA,...,"nm0004056,nm0028764,nm0149290,nm0123194",tt0401729,10,nm0606640,producer,producer,NaN,tt0401729,6.6,241792.0
3,France,NaN,NaN,NaN,NaN,52 min,tt10010134,1,Versailles Rediscovered - The Sun King's Vanis...,NaN,...,"nm2205937,nm1355962,nm10542772,nm0873469",tt10010134,10,nm0674930,editor,NaN,NaN,tt10010134,7.0,5.0
4,Australia,NaN,NaN,NaN,NaN,94 min,tt10027708,1,Miguelito - Canto a Borinquen,NaN,...,nm3495624,tt10027708,10,nm7089722,editor,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25144,USA,NaN,NaN,NaN,NaN,69 min,tt3328050,1,Moving from Emptiness: The Life and Art of a Z...,US,...,nm1893060,tt3328050,1,nm0366799,director,co-director,NaN,tt3328050,8.5,11.0
25145,Croatia,"EUR3,000",NaN,NaN,NaN,70 min,tt3329964,1,Glazbena kutija,HR,...,NaN,tt3329964,10,nm1801528,actress,NaN,"[""Teacher""]",tt3329964,8.0,29.0
25146,USA,NaN,NaN,NaN,NaN,73 min,tt3330812,1,Night Terrors,US,...,"nm2581157,nm3328847",tt3330812,10,nm3329993,editor,NaN,NaN,NaN,NaN,NaN
25147,Costa Rica,NaN,NaN,NaN,NaN,125 min,tt3331178,1,La Musa es la Montaña,CR,...,nm4889713,tt3331178,1,nm4889713,director,NaN,NaN,NaN,NaN,NaN


In [19]:
print(df.columns)
print(df.isna().sum())

Index(['Country', 'Budget', 'Opening Weekend USA', 'Gross USA',
       'Cumulative Worldwide Gross', 'Runtime', 'title_id', 'ordering_x',
       'title', 'region', 'language', 'types', 'attributes',
       'is_original_title', 'tconst_x', 'primary_title', 'original_title',
       'start_year', 'runtime_minutes', 'genres', 'tconst_y', 'directors',
       'writers', 'tconst_x', 'ordering_y', 'nconst', 'category', 'job',
       'characters', 'tconst_y', 'averagerating', 'numvotes'],
      dtype='object')
Country                          32
Budget                        17400
Opening Weekend USA           23348
Gross USA                     23257
Cumulative Worldwide Gross    20143
Runtime                        6378
title_id                          0
ordering_x                        0
title                             0
region                         2871
language                      23521
types                         18605
attributes                    24547
is_original_title        

In [20]:
# dropping unnecessary columns
df.drop(['ordering_x', 'language', 'attributes', 'original_title', 'Runtime'], axis=1, inplace=True)

In [23]:
df.head(500)

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,title_id,title,region,types,is_original_title,...,writers,tconst_x,ordering_y,nconst,category,job,characters,tconst_y,averagerating,numvotes
0,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",tt0369610,Джурасик свят,BG,NaN,0.0,...,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",tt0369610,10,nm0189777,producer,producer,NaN,tt0369610,7.0,539338.0
1,UK,NaN,NaN,NaN,NaN,tt0381957,Cloud Cuckoo Land,GB,NaN,0.0,...,"nm1458371,nm1461088",tt0381957,10,nm1108038,composer,NaN,NaN,tt0381957,7.6,23.0
2,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",tt0401729,Джон Картер: Мiж двох свiтiв,UA,NaN,0.0,...,"nm0004056,nm0028764,nm0149290,nm0123194",tt0401729,10,nm0606640,producer,producer,NaN,tt0401729,6.6,241792.0
3,France,NaN,NaN,NaN,NaN,tt10010134,Versailles Rediscovered - The Sun King's Vanis...,NaN,original,1.0,...,"nm2205937,nm1355962,nm10542772,nm0873469",tt10010134,10,nm0674930,editor,NaN,NaN,tt10010134,7.0,5.0
4,Australia,NaN,NaN,NaN,NaN,tt10027708,Miguelito - Canto a Borinquen,NaN,original,1.0,...,nm3495624,tt10027708,10,nm7089722,editor,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Portugal,NaN,NaN,NaN,NaN,tt2027262,Com Um Pouco de Fé,PT,NaN,0.0,...,nm1796990,tt2027262,10,nm4711841,actor,NaN,"[""Fuinha""]",NaN,NaN,NaN
496,USA,NaN,NaN,NaN,NaN,tt2032524,Rostropovich and Shostakovich. Masterclass of ...,NaN,original,1.0,...,NaN,tt2032524,1,nm4576832,director,NaN,NaN,NaN,NaN,NaN
497,India,"INR10,000,000",NaN,NaN,NaN,tt2033264,Hair is Falling: A Serious Comedy Film,IN,NaN,0.0,...,"nm4578469,nm4781188,nm4781196,nm4781170,nm4768265",tt2033264,10,nm4781164,producer,producer,NaN,tt2033264,5.4,30.0
498,UK,"GBP220,000",NaN,NaN,NaN,tt2033298,Leslie Howard: The Man Who Gave a Damn,NaN,original,1.0,...,"nm0294301,nm0664041,nm4578741",tt2033298,10,nm2325989,cinematographer,NaN,NaN,tt2033298,8.3,190.0
